<a href="https://colab.research.google.com/github/woodongk/news-comments_emotion_classification-CNN-tensorflow/blob/master/Text_CNN_%EB%AA%A8%EB%8D%B8_Load_%26_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
word2vec_model_path = "/content/drive/MyDrive/Dataset 및 Project 정리/daum news and comments (2016 - 2017)/word_embedding_model/word2vec300/0710 w2v_model"
data_path = "/content/drive/MyDrive/Dataset 및 Project 정리/daum news and comments (2016 - 2017)/final_data/comment_with_emo_over0.5.pkl"

In [8]:
import pandas as pd
import numpy as np
import pickle
import codecs
import time
import os

from itertools import chain
from collections import Counter

import sys
import tensorflow as tf
from tensorflow import keras

In [9]:
tf.__version__

'2.4.1'

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from keras.models import load_model

# 모델 불러오기

In [11]:
SAVE_MODEL_PATH = "/content/drive/MyDrive/Dataset 및 Project 정리/daum news and comments (2016 - 2017)/model/20210502 textcnn (epoch 50)"
SAVE_WEIGHT_PATH = "/content/drive/MyDrive/Dataset 및 Project 정리/daum news and comments (2016 - 2017)/model/20210502 textcnn (epoch 50)/check_point"

In [12]:
new_model = tf.keras.models.load_model(SAVE_MODEL_PATH)

# 모델 추론

In [13]:
FC_DATA_PATH = "/content/drive/MyDrive/2020/Lab/Project/[2020.01~] FactCheck News/Data/Result_ver2/00 tokenized/comments_tokenized_wo_exception_emotion (332690) v3.pkl"

In [ ]:
fc_data = pd.read_pickle(FC_DATA_PATH)
comment_token_list = fc_data['tk_cmts'].tolist()
comment_token_list[:5]

# input 데이터를 토큰화 및 패딩시키기

In [28]:
vocab_size = 30000
MAX_SEQUENCE_LENGTH = 20
class_names = np.array(['angry', 'disgust', 'fear', 'happy', 'sad', 'surprised'])

def predict_new_data(model,
                     token_lst, 
                     vocab_size, 
                     MAX_SEQUENCE_LENGTH, 
                     class_names):
  tokenizer = Tokenizer(num_words=vocab_size)
  tokenizer.fit_on_texts(token_lst)

  x_sequence = tokenizer.texts_to_sequences(token_lst)
  x_padded = pad_sequences(x_sequence, maxlen=MAX_SEQUENCE_LENGTH)

  y_proba = model.predict(x_padded)
  y_pred = np.argmax(y_proba,axis=1)
  
  return class_names[y_pred]

In [30]:
predict_results = predict_new_data(new_model, comment_token_list[:50], vocab_size, MAX_SEQUENCE_LENGTH, class_names)
predict_results

array(['sad', 'fear', 'disgust', 'angry', 'fear', 'disgust', 'fear',
       'surprised', 'disgust', 'sad', 'happy', 'angry', 'happy',
       'disgust', 'disgust', 'angry', 'surprised', 'sad', 'angry',
       'disgust', 'fear', 'angry', 'fear', 'fear', 'surprised', 'angry',
       'sad', 'sad', 'sad', 'fear', 'angry', 'angry', 'surprised',
       'happy', 'fear', 'fear', 'sad', 'sad', 'sad', 'fear', 'happy',
       'happy', 'fear', 'sad', 'sad', 'angry', 'disgust', 'angry',
       'happy', 'fear'], dtype='<U9')

In [34]:
for i in range(50):
  print(comment_token_list[i],predict_results[i])

['솔직히', '고발'] sad
['쉿', '조용', '그', '서울대', '의대', '출신', '서울대', '의대', '교수', '는', '특혜', '잘못', '귀걸이', '착용', '경쟁', '없이', '채용', '날짜', '지나다', '제출', '마', '고마', '아들', '먼저', '다'] fear
['댓글', '쓰다', '지우다', '는', '사람', '많다', '귀찮다', '천', '가까이', '자기', '글', '게이버', '썩다', '은', '내', '에', '코', '마비'] disgust
['진짜', '편파', 'ㅋ'] angry
['안철수', '부인', '팩', '트', '체크', '의혹', '중심', '일부', '사실', '다르다', '마치', '일부', '특혜', '몰아가다', '문재인', '아들', '문', '해명', '훨씬', '길다', '해명', '중심', '편들다', '기', '너무', '티', '너무', '편파'] fear
['지원', '합격', '지원', '합격', '도진', '개진', '의미', '읍', '다', '예사말', '지원', '그', '짓', '말', '국민', '개돼지', '보이다'] disgust
['위', '뉴스', '함께', '보다', '자료', '문재인', '아들', '특혜', '문제', '해명', '자료', '확', '하다', '판단', '알다'] fear
['아버지', '직속', '부하', '있다', '는', '공기업', '공채', '어떻게', '하루', '인터넷', '공지', '띄다', '그', '아들', '알다', '응시', '경쟁', '말장난', '하다', '전문가', '한', '모집', '한', '응시', '합격', '누구', '개돼지', '아', '나누다', '뽑다', '취업', '시험', '그런', '기회', '있다', '그', '사람', '아들'] surprised
['확증', '없다', '지원', '뽑다', '는', '공채', '있다', '이재명', '안', '되다', '문재인', '찍